
# Dyadic Δ Scan Notebook (v3, strict‑pin)

*Resolves the NumPy binary‑compat error by forcing NumPy 1.26, SciPy 1.11, pandas 1.5, seaborn 0.13.*

---

This Colab notebook measures

\[
\Delta_C(n)=\min_i\bigl|C-(i/2^n)^2\bigr|
\]

for each constant up to `n_max` bits and extracts the slope ≈−2, phase lock, and κ plateau.


In [1]:
# -------- Dependency block (strict versions) -----------
!pip -q install --no-cache-dir --upgrade \
    numpy==1.26.4 \
    pandas==1.5.3 \
    scipy==1.11.4 \
    matplotlib==3.8.3 \
    seaborn==0.13.2 \
    mpmath==1.3.0 \
    tqdm -U

import mpmath as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from scipy import stats

mp.mp.dps = 500   # 1700‑bit precision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 77.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pan

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# ==== Parameters ==============================================================
n_max = 200  # bits of precision

CONSTANTS = {
    "alpha"        : mp.mpf("1")/mp.mpf("137.035999"),
    "phi"          : (1 + mp.sqrt(5)) / 2,
    "pi"           : mp.pi,
    "pi/e"         : mp.pi / mp.e,
    "sqrt3/pi"     : mp.sqrt(3) / mp.pi,
    "sqrt2"        : mp.sqrt(2),
    "e"            : mp.e,

    # towers
    "e/pi tower"   : mp.mpf("0.880367778981734621826749"),
    "pi/e tower"   : mp.mpf("1.187523635359249905438407"),
    "lil phi tower": mp.mpf("0.710439287156503188669345"),
    "1/2 tower"    : mp.mpf("0.641185744504985984486200"),
}
# ==============================================================================


In [ ]:
def best_delta(C, n):
    scale = mp.mpf(1) << n
    est = int(mp.nint(mp.sqrt(C) * scale))
    best, best_i = mp.mpf('inf'), None
    for i in range(est-3, est+4):
        d = abs((i/scale)**2 - C)
        if d < best:
            best, best_i = d, i
    return best, best_i

def sweep(constants, n_max):
    rows = []
    for name, C in constants.items():
        for n in tqdm(range(1, n_max+1), desc=f"{name:14s}"):
            d, i_star = best_delta(C, n)
            rows.append({
                "constant": name,
                "n": n,
                "delta": d,
                "log10_delta": mp.log10(d),
                "phase": i_star % 4,
                "kappa_n": (2**(2*n))*d,
            })
    return pd.DataFrame(rows)

df = sweep(CONSTANTS, n_max)
df.to_pickle("delta_scan.pkl")
print("Saved → delta_scan.pkl")


In [ ]:
# Δ decay plot
plt.figure(figsize=(8,5))
for name, grp in df.groupby("constant"):
    plt.plot(grp["n"], grp["log10_delta"], label=name)
plt.xlabel("n (bits)"); plt.ylabel("log10 Δ")
plt.title("Δ decay across constants"); plt.legend(bbox_to_anchor=(1.05,1))
plt.tight_layout(); plt.show()

# Phase histogram
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="phase", hue="constant")
plt.title("Residue class of best numerator (i_* mod 4)")
plt.tight_layout(); plt.show()


In [ ]:
print("=== Slope diagnostics ===")
for name, grp in df.groupby("constant"):
    slope, *_ , r, p, _ = stats.linregress(grp["n"], np.log2(grp["delta"]))
    print(f"{name:14s} slope={slope:7.3f}  R²={r**2:6.4f}  p={p:.1e}")

print("\n=== κ plateau ===")
for name, grp in df.groupby("constant"):
    plateau = grp[grp["n"] > n_max-20]["kappa_n"].median()
    print(f"{name:14s} κ≈{plateau:.3e}")


## Sandbox

In [ ]:
# κ_n trajectory for 'alpha'
sub = df[df['constant']=='alpha']
plt.plot(sub['n'], sub['kappa_n'])
plt.title('κ_n trajectory (alpha)')
plt.xlabel('n'); plt.ylabel('κ_n')
plt.show()
